In [3]:
from itertools import combinations 
  
def rSubset(arr, r): 
  
    # return list of all subsets of length r 
    # to deal with duplicate subsets use  
    # set(list(combinations(arr, r))) 
    return list(combinations(arr, r)) 
  
# Driver Function 
if __name__ == "__main__": 
    arr = [1, 2, 3, 4] 
    rcomb_all = []
    for r in range(len(arr)):
        rcomb = rSubset(arr, r+1) 
        rcomb_all.extend(rcomb)
rcomb_all

[(1,),
 (2,),
 (3,),
 (4,),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4),
 (1, 2, 3),
 (1, 2, 4),
 (1, 3, 4),
 (2, 3, 4),
 (1, 2, 3, 4)]

In [11]:
def colCombination(arr, rmin, rmax=100):
    rcomb_all = []
    if rmax > len(arr):
        rmax = len(arr)
    for r in range(rmin, rmax, 1):
        rcomb = rSubset(arr, r) 
        rcomb_all.extend(rcomb)
    return rcomb_all
arr = [1,2,3,4,5,6,7]
colCombination(arr, 3,5)

[(1, 2, 3),
 (1, 2, 4),
 (1, 2, 5),
 (1, 2, 6),
 (1, 2, 7),
 (1, 3, 4),
 (1, 3, 5),
 (1, 3, 6),
 (1, 3, 7),
 (1, 4, 5),
 (1, 4, 6),
 (1, 4, 7),
 (1, 5, 6),
 (1, 5, 7),
 (1, 6, 7),
 (2, 3, 4),
 (2, 3, 5),
 (2, 3, 6),
 (2, 3, 7),
 (2, 4, 5),
 (2, 4, 6),
 (2, 4, 7),
 (2, 5, 6),
 (2, 5, 7),
 (2, 6, 7),
 (3, 4, 5),
 (3, 4, 6),
 (3, 4, 7),
 (3, 5, 6),
 (3, 5, 7),
 (3, 6, 7),
 (4, 5, 6),
 (4, 5, 7),
 (4, 6, 7),
 (5, 6, 7),
 (1, 2, 3, 4),
 (1, 2, 3, 5),
 (1, 2, 3, 6),
 (1, 2, 3, 7),
 (1, 2, 4, 5),
 (1, 2, 4, 6),
 (1, 2, 4, 7),
 (1, 2, 5, 6),
 (1, 2, 5, 7),
 (1, 2, 6, 7),
 (1, 3, 4, 5),
 (1, 3, 4, 6),
 (1, 3, 4, 7),
 (1, 3, 5, 6),
 (1, 3, 5, 7),
 (1, 3, 6, 7),
 (1, 4, 5, 6),
 (1, 4, 5, 7),
 (1, 4, 6, 7),
 (1, 5, 6, 7),
 (2, 3, 4, 5),
 (2, 3, 4, 6),
 (2, 3, 4, 7),
 (2, 3, 5, 6),
 (2, 3, 5, 7),
 (2, 3, 6, 7),
 (2, 4, 5, 6),
 (2, 4, 5, 7),
 (2, 4, 6, 7),
 (2, 5, 6, 7),
 (3, 4, 5, 6),
 (3, 4, 5, 7),
 (3, 4, 6, 7),
 (3, 5, 6, 7),
 (4, 5, 6, 7)]

In [129]:
from sklearn.base import TransformerMixin, BaseEstimator
class DateTransformer(TransformerMixin, BaseEstimator):
    """Extracts features from datetime column
    
    Returns:
      hour: hour
      day: Between 1 and the number of days in the month
      month: Between 1 and 12 inclusive.
      year: four-digit year
      weekday: day of the week as an integer. Mon=0 and Sun=6
   """
    def fit(self, x, y=None):
        return self
    def transform(self, x, y=None):
        result = pd.DataFrame(x, columns=['date_hour'])
        result['hour'] = [dt.hour for dt in result['date_hour']]
        result['day'] = [dt.day for dt in result['date_hour']]
        result['month'] = [dt.month for dt in result['date_hour']]
        result['year'] = [dt.year for dt in result['date_hour']]
        result['weekday'] = [dt.weekday() for dt in 
                             result['date_hour']]
        return result[['hour', 'day', 'month', 'year', 'weekday']]
    
    def get_feature_names(self):
        return ['hour','day', 'month', 'year', 'weekday']

In [163]:
class SubsetDataFrame(TransformerMixin, BaseEstimator):
    """Extracts requested columns from dataframe
    
    Returns:
      x[columns]: subset of the dataframe
    """
    def __init__(self, columns):
        self.columns = columns
        
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, x, y=None):
        remaining_cols = set(self.columns) - set(x.columns)
        if remaining_cols != set():
            raise Exception(f"Columns : {remaining_cols} not in the dataframe")
        else:
            return x[self.columns]
    
    def get_feature_names(self):
        return self.columns
    

In [125]:
x = set([4])
print(x)
x == set()

{4}


False

In [266]:
class NumericColumnStatistics(TransformerMixin, BaseEstimator):
    """Extracts statistics for numerical features 
    Returns:
      hour: hour
      day: Between 1 and the number of days in the month
      month: Between 1 and 12 inclusive.
      year: four-digit year
      weekday: day of the week as an integer. Mon=0 and Sun=6
   """
    def __init__(self, df, groupby_col = None, stat='all'):
        self.groupby_col = groupby_col
        self.stat = stat
        self.new_cols = None
        self.df = df
        self.implemented = ['min','max','median','mean','all']
        self.features=1
        
    def fit(self, x, y=None):
        return self
    
    def transform(self, x, y=None):
        self.features = x.shape[1]
        x = x.copy()
        x_df_numpy = np.column_stack([x, self.df[self.groupby_col].values])
        
        print(x_df_numpy[:5])
        
        num_cols = [f'num_{i}' for i in range(self.features)]
        print(f'New col: {num_cols}')
        all_cols = num_cols + [self.groupby_col]
        
        x_df = pd.DataFrame(x_df_numpy, columns=all_cols)
        
        for col in num_cols:
            x_df[col] = x_df[col].astype(np.float)
        
        if self.stat not in self.implemented:
            raise Exception(f"Only following stats are implemented: {self.implemented}")        
        else:
            x_transformed = self._stat_calc(x_df, self.stat)
    
        return x_transformed[self.new_cols]
    
    def _stat_calc(self, df, stat=None):
        print(df.info())
        print(f'groupby coll: {self.groupby_col}')
        
        
        if stat=='min':
            if self.groupby_col != None:
                agg_df = df.groupby(self.groupby_col).min()
                self.new_cols = [f'{col}_{self.stat}' for col in agg_df.columns]
                agg_df.columns = self.new_cols
                agg_df = agg_df.reset_index()
                df = df.merge(agg_df, on=self.groupby_col, how='left')
        if stat=='mean':
            if self.groupby_col != None:
                agg_df = df.groupby(self.groupby_col).mean()
                self.new_cols = [f'{col}_{self.stat}' for col in agg_df.columns]
                agg_df.columns = self.new_cols
                agg_df = agg_df.reset_index()
                df = df.merge(agg_df, on=self.groupby_col, how='left')    
        if stat=='max':
            if self.groupby_col != None:
                agg_df = df.groupby(self.groupby_col).max()
                self.new_cols = [f'{col}_{self.stat}' for col in agg_df.columns]
                agg_df.columns = self.new_cols
                agg_df = agg_df.reset_index()
                df = df.merge(agg_df, on=self.groupby_col, how='left')
    
        if stat=='median':
            if self.groupby_col != None:
                agg_df = df.groupby(self.groupby_col).median()
                self.new_cols = [f'{col}_{self.stat}' for col in agg_df.columns]
                agg_df.columns = self.new_cols
                agg_df = agg_df.reset_index()
                df = df.merge(agg_df, on=self.groupby_col, how='left')
        if stat=='all':
            if self.groupby_col != None:
                agg_df = df.groupby(self.groupby_col).agg(['mean','min','max','count'])
                self.new_cols = [f'{col}_{self.stat}' for col in agg_df.columns]
                agg_df.columns = self.new_cols
                agg_df = agg_df.reset_index()
                df = df.merge(agg_df, on=self.groupby_col, how='left')
        return df
    
    def get_feature_names(self):
        return self.new_cols

In [ ]:
class CategoricalColumnStatistics(TransformerMixin, BaseEstimator):
    """Extracts statistics for categorical features 
    Returns:
   """
    def __init__(self, df, target=None, groupby_col = None, stat='all'):
        self.groupby_col = groupby_col
        self.stat = stat
        self.new_cols = None
        self.df = df
        self.implemented = ['count']
        self.features=1
        
    def fit(self, x, y=None):
        return self
    
    def transform(self, x, y=None):
        self.features = x.shape[1]
        x = x.copy()
        x_df_numpy = np.column_stack([x, self.df[self.groupby_col].values])
        
        print(x_df_numpy[:5])
        
        num_cols = [f'num_{i}' for i in range(self.features)]
        print(f'New col: {num_cols}')
        all_cols = num_cols + [self.groupby_col]
        
        x_df = pd.DataFrame(x_df_numpy, columns=all_cols)
        
        for col in num_cols:
            x_df[col] = x_df[col].astype(np.float)
        
        if self.stat not in self.implemented:
            raise Exception(f"Only following stats are implemented: {self.implemented}")        
        else:
            x_transformed = self._stat_calc(x_df, self.stat)
    
        return x_transformed[self.new_cols]
    
    def _stat_calc(self, df, stat=None):
        print(df.info())
        print(f'groupby coll: {self.groupby_col}')
        
        
        if stat=='min':
            if self.groupby_col != None:
                agg_df = df.groupby(self.groupby_col).min()
                self.new_cols = [f'{col}_{self.stat}' for col in agg_df.columns]
                agg_df.columns = self.new_cols
                agg_df = agg_df.reset_index()
                df = df.merge(agg_df, on=self.groupby_col, how='left')
        if stat=='mean':
            if self.groupby_col != None:
                agg_df = df.groupby(self.groupby_col).mean()
                self.new_cols = [f'{col}_{self.stat}' for col in agg_df.columns]
                agg_df.columns = self.new_cols
                agg_df = agg_df.reset_index()
                df = df.merge(agg_df, on=self.groupby_col, how='left')    
        if stat=='max':
            if self.groupby_col != None:
                agg_df = df.groupby(self.groupby_col).max()
                self.new_cols = [f'{col}_{self.stat}' for col in agg_df.columns]
                agg_df.columns = self.new_cols
                agg_df = agg_df.reset_index()
                df = df.merge(agg_df, on=self.groupby_col, how='left')
    
        if stat=='median':
            if self.groupby_col != None:
                agg_df = df.groupby(self.groupby_col).median()
                self.new_cols = [f'{col}_{self.stat}' for col in agg_df.columns]
                agg_df.columns = self.new_cols
                agg_df = agg_df.reset_index()
                df = df.merge(agg_df, on=self.groupby_col, how='left')
        if stat=='all':
            if self.groupby_col != None:
                agg_df = df.groupby(self.groupby_col).agg(['mean','min','max','count'])
                self.new_cols = [f'{col}_{self.stat}' for col in agg_df.columns]
                agg_df.columns = self.new_cols
                agg_df = agg_df.reset_index()
                df = df.merge(agg_df, on=self.groupby_col, how='left')
        return df
    
    def get_feature_names(self):
        return self.new_cols

In [18]:
import numpy as np
import pandas as pd
L = np.random.random((100,5))
L.min(axis=0)

array([0.0004014 , 0.00238867, 0.01417285, 0.00953453, 0.00369286])

In [19]:
aa_milne_arr = ['pooh', 'rabbit', 'piglet', 'Christopher']
aa_milne_ndarr = np.random.choice(aa_milne_arr, 100, p=[0.5, 0.1, 0.1, 0.3])
aa_milne_ndarr

array(['pooh', 'pooh', 'Christopher', 'pooh', 'pooh', 'pooh',
       'Christopher', 'pooh', 'pooh', 'pooh', 'pooh', 'pooh', 'pooh',
       'pooh', 'rabbit', 'pooh', 'pooh', 'pooh', 'pooh', 'Christopher',
       'pooh', 'Christopher', 'Christopher', 'pooh', 'pooh', 'pooh',
       'pooh', 'piglet', 'Christopher', 'rabbit', 'Christopher',
       'Christopher', 'pooh', 'pooh', 'piglet', 'piglet', 'Christopher',
       'piglet', 'piglet', 'Christopher', 'rabbit', 'rabbit', 'rabbit',
       'Christopher', 'Christopher', 'Christopher', 'pooh', 'pooh',
       'pooh', 'piglet', 'pooh', 'piglet', 'piglet', 'piglet', 'pooh',
       'pooh', 'piglet', 'piglet', 'piglet', 'pooh', 'Christopher',
       'pooh', 'Christopher', 'pooh', 'rabbit', 'pooh', 'pooh',
       'Christopher', 'pooh', 'rabbit', 'piglet', 'piglet', 'pooh',
       'pooh', 'pooh', 'piglet', 'pooh', 'piglet', 'rabbit',
       'Christopher', 'Christopher', 'piglet', 'pooh', 'pooh', 'pooh',
       'pooh', 'Christopher', 'pooh', 'Christo

In [21]:
L_ = np.column_stack((aa_milne_ndarr, L))

In [197]:
data_df = pd.DataFrame(L_, columns=['category', 'num1','num2','num3','num4','num5'])
num_cols = ['num1','num2','num3','num4','num5']

In [198]:
for col in num_cols:
    data_df[col] = data_df[col].astype(np.float)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   category  100 non-null    object 
 1   num1      100 non-null    float64
 2   num2      100 non-null    float64
 3   num3      100 non-null    float64
 4   num4      100 non-null    float64
 5   num5      100 non-null    float64
dtypes: float64(5), object(1)
memory usage: 4.8+ KB


In [164]:
num3_selector = SubsetDataFrame(['num2','num4','num1'])
cat_num2_selector = SubsetDataFrame(['num2','category'])
cat_num4_selector = SubsetDataFrame(['num2','category'])

data_num3_df = num3_selector.fit_transform(data_df)
num3_selector.get_feature_names()
cat_num2_selector.get_feature_names()

['num2', 'category']

In [83]:
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.preprocessing import FunctionTransformer

In [74]:
num_selector= make_column_selector(dtype_include=np.float)
num_selector(data_df)

['num1', 'num2', 'num3', 'num4', 'num5']

In [306]:
cat_selector = make_column_transformer(
    ('passthrough', make_column_selector(dtype_include=['object','bool', np.int])), remainder='drop'
)


num_selector = make_column_transformer(
    ('passthrough', make_column_selector(dtype_include=np.float)), remainder='drop'
)

In [137]:
combined = FeatureUnion([
    ('numerical_df', num_selector),
    ('categorical_df', cat_selector),
])
combined.fit_transform(data_df)[:5]

array([[0.4587857843536842, 0.18815731611629272, 0.3879015273014733,
        0.13372282746462294, 0.6465516469980312, 'pooh'],
       [0.6818258668249646, 0.9707033956756659, 0.243337406720473,
        0.9450546129398608, 0.7740682665155234, 'pooh'],
       [0.36804660223116203, 0.21852695871778804, 0.7540796872182675,
        0.4655301812192645, 0.7043660146474516, 'Christopher'],
       [0.8690683213428335, 0.056363860512781416, 0.7207150531424162,
        0.1473129247921492, 0.29983539745466936, 'pooh'],
       [0.4829679747676281, 0.7936730817926014, 0.48393563528249317,
        0.3320090089654918, 0.09682161588258031, 'pooh']], dtype=object)

In [196]:
def set_index(df, col):
    df = df.copy()
    df.index = df[col]
    return df.drop(columns=col)
def reset_index_col(df):
    return df.reset_index()

category_index_transformer = FunctionTransformer(set_index, kw_args={'col':'category'})
category_indexset.fit_transform(data_df).head()
reset_index_transformer = FunctionTransformer(reset_index_col)
                                        

In [258]:
min_stats_estimator = NumericStatistics(data_df, groupby_col='category',stat='min')
mean_stats_estimator = NumericStatistics(data_df, groupby_col='category',stat='mean')

In [262]:
min_stats_estimator2 = NumericStatistics(data_df, groupby_col='category',stat='min')
mean_stats_estimator2 = NumericStatistics(data_df, groupby_col='category',stat='mean')

mean_stats_estimator2.fit_transform(data_df['num2'].values.reshape(-1,1))

[[0.18815731611629272 'pooh']
 [0.9707033956756659 'pooh']
 [0.21852695871778804 'Christopher']
 [0.056363860512781416 'pooh']
 [0.7936730817926014 'pooh']]
New col: ['num_0']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   num_0     100 non-null    float64
 1   category  100 non-null    object 
dtypes: float64(1), object(1)
memory usage: 1.7+ KB
None
groupby coll: category


,num_0_mean
0,0.506496
1,0.506496
2,0.404739
3,0.506496
4,0.506496
...,...
95,0.404739
96,0.404739
97,0.404739
98,0.393948


In [234]:
data_df.groupby('category').mean()

,num1,num2,num3,num4,num5
category,,,,,
Christopher,0.533858,0.404739,0.585466,0.462954,0.467221
piglet,0.604845,0.489910,0.457293,0.611971,0.513562
pooh,0.520943,0.506496,0.505767,0.592571,0.477971
rabbit,0.392470,0.393948,0.554818,0.348745,0.319915


In [205]:
data_df.head()

,category,num1,num2,num3,num4,num5
0,pooh,0.458786,0.188157,0.387902,0.133723,0.646552
1,pooh,0.681826,0.970703,0.243337,0.945055,0.774068
2,Christopher,0.368047,0.218527,0.754080,0.465530,0.704366
3,pooh,0.869068,0.056364,0.720715,0.147313,0.299835
4,pooh,0.482968,0.793673,0.483936,0.332009,0.096822


In [209]:
data2_df = data_df.set_index('category')
data2_df.num1.groupby('category').mean()

category
Christopher    0.533858
piglet         0.604845
pooh           0.520943
rabbit         0.392470
Name: num1, dtype: float64

In [200]:
min_stats_estimator.fit_transform(data_df)

<class 'pandas.core.frame.DataFrame'>


,num1_min,num2_min,num3_min,num4_min,num5_min
0,0.046837,0.025914,0.014173,0.009535,0.016807
1,0.046837,0.025914,0.014173,0.009535,0.016807
2,0.071098,0.040229,0.055135,0.018288,0.003693
3,0.046837,0.025914,0.014173,0.009535,0.016807
4,0.046837,0.025914,0.014173,0.009535,0.016807
...,...,...,...,...,...
95,0.071098,0.040229,0.055135,0.018288,0.003693
96,0.071098,0.040229,0.055135,0.018288,0.003693
97,0.071098,0.040229,0.055135,0.018288,0.003693
98,0.069233,0.002389,0.057729,0.020220,0.030551


In [214]:
data_stats = data_df.groupby('category').agg(['mean','count','min','max','median'])
data_stats.head()

num1                                          num2        \
                 mean count       min       max    median      mean count   
category                                                                    
Christopher  0.533858    26  0.071098  0.999538  0.536695  0.404739    26   
piglet       0.604845    19  0.000401  0.971149  0.716010  0.489910    19   
pooh         0.520943    46  0.046837  0.940017  0.548500  0.506496    46   
rabbit       0.392470     9  0.069233  0.910316  0.402400  0.393948     9   

                                           ...      num4                  \
                  min       max    median  ...      mean count       min   
category                                   ...                             
Christopher  0.040229  0.936332  0.284556  ...  0.462954    26  0.018288   
piglet       0.058458  0.925025  0.483230  ...  0.611971    19  0.096880   
pooh         0.025914  0.970703  0.524665  ...  0.592571    46  0.009535   
rabbit       0.002389  0.906647  0.292476  ...  0.348745     9  0.020220   

                                     num5                                      
                  max    median      mean count       min       max    median  
category                                                                       
Christopher  0.980167  0.532532  0.467221    26  0.003693  0.998744  0.446900  
piglet       0.980688  0.644871  0.513562    19  0.033732  0.916945  0.479204  
pooh         0.996677  0.634940  0.477971    46  0.016807  0.967459  0.480816  
rabbit       0.874167  0.242895  0.319915     9  0.030551  0.795247  0.292552  

[4 rows x 25 columns]

In [217]:
stat_cols = []
for col1, col2 in data_stats.columns:
    stat_cols.append(col1+'_'+col2)
data_stats.columns = stat_cols
data_stats.head()

,num1_mean,num1_count,num1_min,num1_max,num1_median,num2_mean,num2_count,num2_min,num2_max,num2_median,...,num4_mean,num4_count,num4_min,num4_max,num4_median,num5_mean,num5_count,num5_min,num5_max,num5_median
category,,,,,,,,,,,,,,,,,,,,,
Christopher,0.533858,26,0.071098,0.999538,0.536695,0.404739,26,0.040229,0.936332,0.284556,...,0.462954,26,0.018288,0.980167,0.532532,0.467221,26,0.003693,0.998744,0.446900
piglet,0.604845,19,0.000401,0.971149,0.716010,0.489910,19,0.058458,0.925025,0.483230,...,0.611971,19,0.096880,0.980688,0.644871,0.513562,19,0.033732,0.916945,0.479204
pooh,0.520943,46,0.046837,0.940017,0.548500,0.506496,46,0.025914,0.970703,0.524665,...,0.592571,46,0.009535,0.996677,0.634940,0.477971,46,0.016807,0.967459,0.480816
rabbit,0.392470,9,0.069233,0.910316,0.402400,0.393948,9,0.002389,0.906647,0.292476,...,0.348745,9,0.020220,0.874167,0.242895,0.319915,9,0.030551,0.795247,0.292552


In [247]:
min_stats_estimator2.groupby_col

'category'

In [283]:
import category_encoders as ce

# encoder = ce.BackwardDifferenceEncoder(cols=[...])
# encoder = ce.BaseNEncoder(cols=[...])
# encoder = ce.BinaryEncoder(cols=[...])
# encoder = ce.CatBoostEncoder(cols=[...])
# encoder = ce.CountEncoder(cols=[...])
# encoder = ce.GLMMEncoder(cols=[...])
# encoder = ce.HashingEncoder(cols=[...])
# encoder = ce.HelmertEncoder(cols=[...])
# encoder = ce.JamesSteinEncoder(cols=[...])
# encoder = ce.LeaveOneOutEncoder(cols=[...])
# encoder = ce.MEstimateEncoder(cols=[...])
# encoder = ce.OneHotEncoder(cols=[...])
# encoder = ce.OrdinalEncoder(cols=[...])
# encoder = ce.SumEncoder(cols=[...])
# encoder = ce.PolynomialEncoder(cols=[...])
# encoder = ce.TargetEncoder(cols=[...])
# encoder = ce.WOEEncoder(cols=[...])

cat_encoder = ce.GLMMEncoder(cols=['category'])
cat_encoder.fit_transform(data_df, data_df.num2)



/Users/aripiralasrinivas/miniconda3/envs/kaggle/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,category,num1,num2,num3,num4,num5
0,0.010429,0.458786,0.188157,0.387902,0.133723,0.646552
1,0.010429,0.681826,0.970703,0.243337,0.945055,0.774068
2,-0.009269,0.368047,0.218527,0.754080,0.465530,0.704366
3,0.010429,0.869068,0.056364,0.720715,0.147313,0.299835
4,0.010429,0.482968,0.793673,0.483936,0.332009,0.096822
...,...,...,...,...,...,...
95,-0.009269,0.273608,0.219361,0.491819,0.201808,0.844004
96,-0.009269,0.513767,0.589216,0.833165,0.609750,0.003693
97,-0.009269,0.559623,0.261420,0.165571,0.585507,0.681755
98,-0.004222,0.167038,0.286717,0.687965,0.242895,0.053057


In [291]:
encoder_choices = np.random.choice(['count','GLM'], 5, p=[0.3,0.7])
print(encoder_choices)
cat_cols = ['category1', 'category2']
category_choices = np.random.choice(cat_cols, 5, p=[0.2,0.8])
print(category_choices)
encoders = [ce.CountEncoder(cols=[col]) if enc_choice=='count' else ce.GLMMEncoder(cols=[col]) 
            for col, enc_choice in zip(category_choices, encoder_choices)]
encoders

['GLM' 'count' 'GLM' 'GLM' 'count']
['category2' 'category1' 'category2' 'category2' 'category1']


[GLMMEncoder(cols=['category2']),
 CountEncoder(cols=['category1'], combine_min_nan_groups=True),
 GLMMEncoder(cols=['category2']),
 GLMMEncoder(cols=['category2']),
 CountEncoder(cols=['category1'], combine_min_nan_groups=True)]

In [300]:
# dict(zip(category_choices, encoder_choices))
encoder_choices
category_choices
for cat, enc in zip(category_choices, encoder_choices):
    print(cat, enc)

category2 GLM
category1 count
category2 GLM
category2 GLM
category1 count


In [264]:
num_processor= ColumnTransformer(transformers=[
    ('min_stats', min_stats_estimator2, ['num1', 'num5']),
    ('mean_stats', mean_stats_estimator2, ['num1', 'num3']),
])
num_processor.fit_transform(data_df)

[[0.4587857843536842 0.6465516469980312 'pooh']
 [0.6818258668249646 0.7740682665155234 'pooh']
 [0.36804660223116203 0.7043660146474516 'Christopher']
 [0.8690683213428335 0.29983539745466936 'pooh']
 [0.4829679747676281 0.09682161588258031 'pooh']]
New col: ['num_0', 'num_1']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   num_0     100 non-null    float64
 1   num_1     100 non-null    float64
 2   category  100 non-null    object 
dtypes: float64(2), object(1)
memory usage: 2.5+ KB
None
groupby coll: category
[[0.4587857843536842 0.3879015273014733 'pooh']
 [0.6818258668249646 0.243337406720473 'pooh']
 [0.36804660223116203 0.7540796872182675 'Christopher']
 [0.8690683213428335 0.7207150531424162 'pooh']
 [0.4829679747676281 0.48393563528249317 'pooh']]
New col: ['num_0', 'num_1']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 t

array([[4.68369920e-02, 1.68066761e-02, 5.20943352e-01, 5.05767382e-01],
       [4.68369920e-02, 1.68066761e-02, 5.20943352e-01, 5.05767382e-01],
       [7.10975358e-02, 3.69285764e-03, 5.33857954e-01, 5.85465655e-01],
       [4.68369920e-02, 1.68066761e-02, 5.20943352e-01, 5.05767382e-01],
       [4.68369920e-02, 1.68066761e-02, 5.20943352e-01, 5.05767382e-01],
       [4.68369920e-02, 1.68066761e-02, 5.20943352e-01, 5.05767382e-01],
       [7.10975358e-02, 3.69285764e-03, 5.33857954e-01, 5.85465655e-01],
       [4.68369920e-02, 1.68066761e-02, 5.20943352e-01, 5.05767382e-01],
       [4.68369920e-02, 1.68066761e-02, 5.20943352e-01, 5.05767382e-01],
       [4.68369920e-02, 1.68066761e-02, 5.20943352e-01, 5.05767382e-01],
       [4.68369920e-02, 1.68066761e-02, 5.20943352e-01, 5.05767382e-01],
       [4.68369920e-02, 1.68066761e-02, 5.20943352e-01, 5.05767382e-01],
       [4.68369920e-02, 1.68066761e-02, 5.20943352e-01, 5.05767382e-01],
       [4.68369920e-02, 1.68066761e-02, 5.20943352e

In [265]:
num_processor.get_feature_names()

['min_stats__num_0_min',
 'min_stats__num_1_min',
 'mean_stats__num_0_mean',
 'mean_stats__num_1_mean']

In [192]:


# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])
pipe = make_pipeline(category_indexset, num_processor, reset_index_transformer)
pipe.fit_transform(data_df)

AttributeError: 'NumericStatistics' object has no attribute 'groupBy_col'

In [187]:
pipe_0 = make_pipeline(category_indexset, )
agg_df = pipe_0.fit_transform(data_df)
agg_df.head()
# num_processor.fit_transform(agg_df)

,num1,num2,num3,num4,num5
category,,,,,
pooh,0.458786,0.188157,0.387902,0.133723,0.646552
pooh,0.681826,0.970703,0.243337,0.945055,0.774068
Christopher,0.368047,0.218527,0.754080,0.465530,0.704366
pooh,0.869068,0.056364,0.720715,0.147313,0.299835
pooh,0.482968,0.793673,0.483936,0.332009,0.096822


In [104]:
data_df.head()

,category,num1,num2,num3,num4,num5
0,pooh,0.458786,0.188157,0.387902,0.133723,0.646552
1,pooh,0.681826,0.970703,0.243337,0.945055,0.774068
2,Christopher,0.368047,0.218527,0.754080,0.465530,0.704366
3,pooh,0.869068,0.056364,0.720715,0.147313,0.299835
4,pooh,0.482968,0.793673,0.483936,0.332009,0.096822


In [173]:
num2_min_trans = make_pipeline(cat_num2_selector, min_stats_estimator)
num4_min_trans = make_pipeline(cat_num4_selector, mean_stats_estimator)
num2_num4 = FeatureUnion([
    ('num2_min', num2_min_trans),
    ('num4_min', num4_min_trans)
])

In [177]:
num2_num4.fit_transform(data_df)[:5]

array([[0.02591387, 0.50649647],
       [0.02591387, 0.50649647],
       [0.04022902, 0.40473877],
       [0.02591387, 0.50649647],
       [0.02591387, 0.50649647]])

In [351]:
data2_df.category.unique()

array(['pooh', 'Christopher', 'rabbit', 'piglet'], dtype=object)

In [178]:
num2_min_trans.fit_transform(data_df[['num2', 'category']]).head()

,num2_min
0,0.025914
1,0.025914
2,0.040229
3,0.025914
4,0.025914


In [349]:
class NumericDescriptiveFeatures(TransformerMixin, BaseEstimator):
    """Extracts statistics for numerical features 
    Returns:
      hour: hour
      day: Between 1 and the number of days in the month
      month: Between 1 and 12 inclusive.
      year: four-digit year
      weekday: day of the week as an integer. Mon=0 and Sun=6
   """
    def __init__(self, groupby_col, stat='all'):
        self.groupby_col = groupby_col
        self.stat = stat
        self.new_cols = None      
        self.implemented = ['min','max','median','mean']
        
    def fit(self, x, y=None):
        num_df = num_selector.fit_transform(x)
        self.num_cols = num_selector.get_feature_names()
        return self
    
    def transform(self, x, y=None):
        x = x.copy()
        if isinstance(self.groupby_col, str):
            all_cols = [self.groupby_col] + self.num_cols
        else:
            all_cols = self.groupby_col + self.num_cols
        x = x[all_cols]
        
        if self.stat not in self.implemented and self.stat !='all':
            raise Exception(f"Only following stats are implemented: {self.implemented}")        
        else:
            x_transformed = self._stat_calc(x)
        return x_transformed
    
    @staticmethod
    def unravel(columns_tuple):
        for col_tuple in columns_tuple:
            cols = [f'{c1}_{c2}' for c1, c2 in columns_tuple]
        return cols
      
    def _stat_calc(self, df):
        if self.stat!='all':
            agg_df = df.groupby(self.groupby_col).agg([self.stat])
        else:
            agg_df = df.groupby(self.groupby_col).agg(self.implemented)
        
        print(agg_df.head())
        print(agg_df.columns)
        
        agg_df.columns = self.unravel(agg_df.columns)
        self.new_cols = agg_df.columns
        agg_df = agg_df.reset_index()
        df = df.merge(agg_df, on=self.groupby_col, how='left')
        return df
    
    def get_feature_names(self):
        return self.new_cols

In [302]:
data2_df = pd.DataFrame(np.column_stack([np.random.choice(aa_milne_arr, 100), data_df.values]))
data2_columns = ['category2']+ list(data_df.columns)
data2_df.columns = data2_columns
data2_df.head()

,category2,category,num1,num2,num3,num4,num5
0,rabbit,pooh,0.458786,0.188157,0.387902,0.133723,0.646552
1,Christopher,pooh,0.681826,0.970703,0.243337,0.945055,0.774068
2,pooh,Christopher,0.368047,0.218527,0.75408,0.46553,0.704366
3,piglet,pooh,0.869068,0.0563639,0.720715,0.147313,0.299835
4,piglet,pooh,0.482968,0.793673,0.483936,0.332009,0.0968216


In [342]:
stats_df = data2_df.groupby(['category2']).agg([ 'max'])
stats_df.columns
def unravel(columns_tuple):
    cols = [f'{c1}_{c2}' for c1, c2 in columns_tuple]
    return cols
stats_df.columns = unravel(stats_df.columns)
stats_df.head()

,category_max,num1_max,num2_max,num3_max,num4_max,num5_max
category2,,,,,,
Christopher,rabbit,0.971149,0.970703,0.979196,0.980688,0.998744
piglet,rabbit,0.869068,0.954104,0.942854,0.991420,0.936751
pooh,rabbit,0.999538,0.873006,0.965364,0.996677,0.967459
rabbit,rabbit,0.940017,0.906647,0.972851,0.921935,0.872264


In [353]:
l = [1,22,3,5]
np.mean(l)

7.75

In [ ]:
cat_selector = make_column_transformer(
    ('passthrough', make_column_selector(dtype_include=['object','bool', np.int])), remainder='drop'
)

num_selector = make_column_transformer(
    ('passthrough', make_column_selector(dtype_include=np.float)), remainder='drop'
)

In [350]:
num_desc_est = NumericDescriptiveFeatures(groupby_col='category2', stat='max')
data2_df_trans = num_desc_est.fit_transform(data2_df)
data2_df_trans.head()

                 num1      num2      num3      num4      num5
                  max       max       max       max       max
category2                                                    
Christopher  0.971149  0.970703  0.979196  0.980688  0.998744
piglet       0.869068  0.954104  0.942854  0.991420  0.936751
pooh         0.999538  0.873006  0.965364  0.996677  0.967459
rabbit       0.940017  0.906647  0.972851  0.921935  0.872264
MultiIndex([('num1', 'max'),
            ('num2', 'max'),
            ('num3', 'max'),
            ('num4', 'max'),
            ('num5', 'max')],
           )


,category2,num1,num2,num3,num4,num5,num1_max,num2_max,num3_max,num4_max,num5_max
0,rabbit,0.458786,0.188157,0.387902,0.133723,0.646552,0.940017,0.906647,0.972851,0.921935,0.872264
1,Christopher,0.681826,0.970703,0.243337,0.945055,0.774068,0.971149,0.970703,0.979196,0.980688,0.998744
2,pooh,0.368047,0.218527,0.754080,0.465530,0.704366,0.999538,0.873006,0.965364,0.996677,0.967459
3,piglet,0.869068,0.056364,0.720715,0.147313,0.299835,0.869068,0.954104,0.942854,0.991420,0.936751
4,piglet,0.482968,0.793673,0.483936,0.332009,0.096822,0.869068,0.954104,0.942854,0.991420,0.936751


In [314]:
for col in num_cols:
    data2_df[col] = data2_df[col].astype(np.float)

In [319]:
cat_selector.fit_transform(data2_df)
cat_selector.get_feature_names()

['category2', 'category']

In [317]:
num_selector.fit_transform(data2_df)
num_selector.get_feature_names()

['num1', 'num2', 'num3', 'num4', 'num5']

In [142]:
data_df.merge(mean_df, on = 'category', how='left').head()

,category,num1,num2,num3,num4,num5,num1_mean,num2_mean,num3_mean,num4_mean,num5_mean
0,pooh,0.458786,0.188157,0.387902,0.133723,0.646552,0.520943,0.506496,0.505767,0.592571,0.477971
1,pooh,0.681826,0.970703,0.243337,0.945055,0.774068,0.520943,0.506496,0.505767,0.592571,0.477971
2,Christopher,0.368047,0.218527,0.754080,0.465530,0.704366,0.533858,0.404739,0.585466,0.462954,0.467221
3,pooh,0.869068,0.056364,0.720715,0.147313,0.299835,0.520943,0.506496,0.505767,0.592571,0.477971
4,pooh,0.482968,0.793673,0.483936,0.332009,0.096822,0.520943,0.506496,0.505767,0.592571,0.477971


In [ ]:
numeric_features = ['age', 'fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['embarked', 'sex', 'pclass']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


In [4]:
import joblib

train_df_optuna_1= joblib.load(f"../input/train_df_optuna_1.0.pkl")
valid_df_optuna_1= joblib.load(f"../input/valid_df_optuna_1.0.pkl")

train_df_1= joblib.load(f"../input/train_df_1.pkl")
valid_df_1= joblib.load(f"../input/valid_df_1.pkl")



In [363]:
print(valid_df_optuna_1.shape, valid_df_1.shape)

(292, 74) (292, 74)


In [365]:
valid_df_optuna_1.equals(valid_df_1)

True

In [4]:
from sklearn.compose import make_column_selector, make_column_transformer
import numpy as np
import pandas as pd

cat_selector = make_column_transformer(
    ('passthrough', make_column_selector(dtype_include=['object','bool', np.int])), remainder='drop'
)

non_cat_selector = make_column_transformer(
    ('drop', make_column_selector(dtype_include=['object','bool', np.int])), remainder='passthrough'
)



non_num_selector = make_column_transformer(
    ('drop', make_column_selector(dtype_include=np.float)), remainder='passthrough'
)

num_selector = make_column_transformer(
    ('passthrough', make_column_selector(dtype_include=np.float)), remainder='drop'
)

In [5]:
train_df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168 entries, 0 to 1167
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LandContour    1168 non-null   int64  
 1   BedroomAbvGr   1168 non-null   int64  
 2   SaleType       1168 non-null   int64  
 3   Foundation     1168 non-null   int64  
 4   YearBuilt      1168 non-null   int64  
 5   CentralAir     1168 non-null   int64  
 6   GarageFinish   1168 non-null   int64  
 7   Functional     1168 non-null   int64  
 8   MSZoning       1168 non-null   int64  
 9   TotRmsAbvGrd   1168 non-null   int64  
 10  RoofStyle      1168 non-null   int64  
 11  LotConfig      1168 non-null   int64  
 12  YearRemodAdd   1168 non-null   int64  
 13  BsmtFinType2   1168 non-null   int64  
 14  MasVnrType     1168 non-null   int64  
 15  BsmtCond       1168 non-null   int64  
 16  RoofMatl       1168 non-null   int64  
 17  HeatingQC      1168 non-null   int64  
 18  PavedDri

In [375]:
pd.DataFrame(non_cat_selector.fit_transform(train_df_1), columns=non_cat_selector.get_feature_names())

,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,PoolArea,MiscVal
0,92.0,10120.0,391.0,740.0,0.0,425.0,1165.0,1203.0,1323.0,0.0,2526.0,1994.000000,844.0,309.0,78.0,0.0,0.0,0.0,0.0
1,82.0,11287.0,340.0,421.0,0.0,386.0,807.0,1175.0,807.0,0.0,1982.0,1989.000000,575.0,0.0,84.0,0.0,0.0,0.0,0.0
2,70.0,7420.0,0.0,417.0,0.0,640.0,1057.0,1057.0,0.0,0.0,1057.0,1977.000000,576.0,0.0,0.0,0.0,0.0,0.0,0.0
3,78.0,10206.0,468.0,33.0,0.0,1530.0,1563.0,1563.0,0.0,0.0,1563.0,2007.000000,758.0,144.0,99.0,0.0,0.0,0.0,0.0
4,61.0,7658.0,412.0,456.0,0.0,1296.0,1752.0,1752.0,0.0,0.0,1752.0,2005.000000,576.0,196.0,82.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,80.0,8000.0,0.0,0.0,0.0,1800.0,1800.0,1800.0,0.0,0.0,1800.0,1978.506164,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1164,50.0,5925.0,0.0,0.0,0.0,600.0,600.0,600.0,368.0,0.0,968.0,1978.506164,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1165,64.0,7314.0,82.0,724.0,0.0,508.0,1232.0,1232.0,0.0,0.0,1232.0,2007.000000,632.0,132.0,0.0,0.0,0.0,0.0,0.0
1166,70.0,7000.0,0.0,969.0,0.0,148.0,1117.0,820.0,527.0,0.0,1347.0,1978.506164,0.0,85.0,0.0,148.0,0.0,0.0,0.0


In [2]:
%cd /Users/aripiralasrinivas/Data Science/repos/mlframework/src

/Users/aripiralasrinivas/Data Science/repos/mlframework/src


In [17]:
from utils import select_dtype_data
select_dtype_data(train_df_1, dtype=[np.float], return_type='numpy')

array([[   40.,  3951.,    76., ...,     0.,     0.,     0.],
       [   69., 13680.,     0., ...,     0.,     0.,     0.],
       [   60.,  9084.,     0., ...,     0.,     0.,     0.],
       ...,
       [   69., 15870.,     0., ...,     0.,     0.,     0.],
       [   94., 12220.,   305., ...,     0.,     0.,     0.],
       [   71.,  9204.,     0., ...,     0.,     0.,     0.]])

In [9]:
from utils import *
len(select_dtype_dataColumns(train_df_1, dtype='NUMERIC'))

19

In [10]:
feature_dtypes_path = get_file_path('Feature_Analysis.csv') 
feature_dtypes_df = read_df(feature_dtypes_path)
feature_dtypes_df.head()

reading the dataframe: ../input/Feature_Analysis.csv


,Unnamed: 0,Name,Feature_Type,Sub_Type,Possible treatments,Possible treatments 2,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,Id,Ignore,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,LotShape,Categorical,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Neighborhood,Categorical,NaN,Reduce the number of categerories,Target mean encoding,NaN,NaN,NaN
3,NaN,OverallQual,Categorical,Ordinal,Target mean encoding,NaN,NaN,NaN,NaN
4,NaN,'RoofMatl',Categorical,NaN,Drop the feature as not much variance,NaN,NaN,NaN,NaN


In [15]:
def clean_col_name(col):
#     print(col)
    col = col.strip()
    if "'" in col:
        col = col.split("'")[1]
    return col


In [16]:
# from utils import clean_col_name
cols_names = feature_dtypes_df[feature_dtypes_df.Sub_Type=='Ordinal'].Name.values
clean_ordinal_cols = [clean_col_name(col) for col in cols_names] 
clean_ordinal_cols

['OverallQual',
 'BsmtHalfBath',
 'KitchenQual',
 'FireplaceQu',
 'GarageQual',
 'OverallCond',
 'FullBath',
 'TotRmsAbvGrd',
 'GarageType',
 'GarageCond',
 'LotConfig',
 'BldgType',
 'BsmtCond',
 'Fireplaces',
 'GarageCars',
 'YearBuilt',
 'BsmtQual',
 'HeatingQC',
 'HalfBath',
 'GarageFinish',
 'PavedDrive',
 'HouseStyle',
 'ExterQual',
 'BsmtExposure',
 'BsmtFullBath']

In [ ]:
import category_encoders as ce
ce.